In [2]:
%load_ext autoreload
%autoreload 2

from querychains import Actor, OpenAiChatEngine, repeat_on_failure, ParsingFailure, Context, FileStorage, start_server
import json

In [3]:

RPS = ["Rock", "Paper", "Scissors"]

def choose_winner(a1, a2):
    if a1 == a2:
        return None
    if a1 == "pass":
        return 1
    if a2 == "pass":
        return 0
    return int((RPS.index(a1) + 1) % 3 == RPS.index(a2)) 

assert choose_winner("Rock", "Paper") == 1
assert choose_winner("Rock", "Scissors") == 0
assert choose_winner("Paper", "Rock") == 0
assert choose_winner("Scissors", "Rock") == 1

In [4]:
storage = FileStorage("data2")

In [6]:

class RockLovingActor(Actor):

    def get_action(self, _prompt):
        return "Rock"


class LmRpsActor(Actor):

    def __init__(self, name, engine, ctx_meta=None,):
        super().__init__(name, ctx_meta)
        self.engine = engine

    def get_action(self, _prompt):
        history = "\n".join(f"Round {i + 1}: the opponent played {op}." for i, op in enumerate(self.observations()))
        parts = ["I'm playing game: Rock Paper Scissors."]
        if history:
            parts.append(f"The history of the game is the following: {history}\n\n")
        parts.append("What should I do in the next turn? Write only exactly one of following words: Rock, Paper, or Scissor as an answer")
        prompt = "\n".join(parts)

        def query():
            result = self.engine.query(prompt)
            result = result.strip()
            if result and result[-1] == ".":
                result = result[:-1]
            if result not in RPS:
                raise ParsingFailure("Invalid action")
            return result
            
        return repeat_on_failure(query, throw_if_fail=False, fail_value="pass")

engine = OpenAiChatEngine(temperature=1)
player1 = LmRpsActor("player1", engine, ctx_meta={"color": "#fcccbb"})
player2 = RockLovingActor("player2", ctx_meta={"color": "lightblue"})

with Context("game", storage=storage) as c:
    wins = [0, 0]
    for i in range(3):
        with Context(f"round {i}"):
            action1 = player1.act("play")
            action2 = player2.act("play")
            print(f"Round: {i} --> {action1} {action2}")
            winner = choose_winner(action1, action2)
            if winner is not None:
                wins[winner] += 1
            player1.observe(action2)
            player2.observe(action1)
c.uid            

Round: 0 --> pass Rock
Round: 1 --> Paper Rock
Round: 2 --> Paper Rock


'2023-06-21T17:57:46-game-9Ee4tF'

In [7]:
storage.start_server(port=5000)

<ServerHandle>

INFO:     Started server process [55188]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5000 (Press CTRL+C to quit)


INFO:     127.0.0.1:58868 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:58868 - "GET /static/js/main.f04c9a55.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:58880 - "GET /static/css/main.073c9b0a.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:58880 - "GET /contexts/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:58880 - "GET /contexts/uid/2023-06-21T17%3A57%3A14-game-jwDS65 HTTP/1.1" 200 OK
INFO:     127.0.0.1:35436 - "GET /contexts/uid/2023-06-21T17%3A57%3A46-game-9Ee4tF HTTP/1.1" 200 OK
INFO:     127.0.0.1:49150 - "GET /contexts/uid/a55d34bf-a6e5-4328-9af2-8d92d8280da5 HTTP/1.1" 200 OK
INFO:     127.0.0.1:46154 - "GET /contexts/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:33506 - "GET /contexts/uid/2023-06-21T18%3A00%3A01-testact-TURzO4 HTTP/1.1" 200 OK
INFO:     127.0.0.1:43982 - "GET /contexts/uid/35422b41-f714-44c0-876c-ca8185936dce HTTP/1.1" 200 OK
INFO:     127.0.0.1:43982 - "GET /contexts/uid/2023-06-21T17%3A57%3A46-game-9Ee4tF HTTP/1.1" 200 OK
INFO:     127.0.0.1:34302 - "GET /contexts/uid/2023-06-

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/home/spirali/.cache/pypoetry/virtualenvs/querychains-RLdrqFKA-py3.10/lib/python3.10/site-packages/uvicorn/protocols/http/httptools_impl.py", line 435, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/home/spirali/.cache/pypoetry/virtualenvs/querychains-RLdrqFKA-py3.10/lib/python3.10/site-packages/uvicorn/middleware/proxy_headers.py", line 78, in __call__
    return await self.app(scope, receive, send)
  File "/home/spirali/.cache/pypoetry/virtualenvs/querychains-RLdrqFKA-py3.10/lib/python3.10/site-packages/fastapi/applications.py", line 276, in __call__
    await super().__call__(scope, receive, send)
  File "/home/spirali/.cache/pypoetry/virtualenvs/querychains-RLdrqFKA-py3.10/lib/python3.10/site-packages/starlette/applications.py", line 122, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/home/spirali/.cache/pypoetry/virtualenvs/querychains-RL

INFO:     127.0.0.1:47644 - "GET /contexts/uid/d43e9aba-f561-49ea-a232-28fbb540df15 HTTP/1.1" 500 Internal Server Error


...

INFO:     127.0.0.1:60494 - "GET /contexts/uid/d43e9aba-f561-49ea-a232-28fbb540df15 HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/home/spirali/.cache/pypoetry/virtualenvs/querychains-RLdrqFKA-py3.10/lib/python3.10/site-packages/uvicorn/protocols/http/httptools_impl.py", line 435, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/home/spirali/.cache/pypoetry/virtualenvs/querychains-RLdrqFKA-py3.10/lib/python3.10/site-packages/uvicorn/middleware/proxy_headers.py", line 78, in __call__
    return await self.app(scope, receive, send)
  File "/home/spirali/.cache/pypoetry/virtualenvs/querychains-RLdrqFKA-py3.10/lib/python3.10/site-packages/fastapi/applications.py", line 276, in __call__
    await super().__call__(scope, receive, send)
  File "/home/spirali/.cache/pypoetry/virtualenvs/querychains-RLdrqFKA-py3.10/lib/python3.10/site-packages/starlette/applications.py", line 122, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/home/spirali/.cache/pypoetry/virtualenvs/querychains-RL

INFO:     127.0.0.1:60504 - "GET /contexts/uid/2023-06-21T16%3A58%3A51-root-PtxMUe HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/home/spirali/.cache/pypoetry/virtualenvs/querychains-RLdrqFKA-py3.10/lib/python3.10/site-packages/uvicorn/protocols/http/httptools_impl.py", line 435, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/home/spirali/.cache/pypoetry/virtualenvs/querychains-RLdrqFKA-py3.10/lib/python3.10/site-packages/uvicorn/middleware/proxy_headers.py", line 78, in __call__
    return await self.app(scope, receive, send)
  File "/home/spirali/.cache/pypoetry/virtualenvs/querychains-RLdrqFKA-py3.10/lib/python3.10/site-packages/fastapi/applications.py", line 276, in __call__
    await super().__call__(scope, receive, send)
  File "/home/spirali/.cache/pypoetry/virtualenvs/querychains-RLdrqFKA-py3.10/lib/python3.10/site-packages/starlette/applications.py", line 122, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/home/spirali/.cache/pypoetry/virtualenvs/querychains-RL

INFO:     127.0.0.1:38688 - "GET /contexts/uid/2023-06-21T16%3A39%3A20-game-BL8gvM HTTP/1.1" 200 OK
INFO:     127.0.0.1:38688 - "GET /contexts/uid/2023-06-21T16%3A39%3A34-game-3oWwPj HTTP/1.1" 200 OK
INFO:     127.0.0.1:38688 - "GET /contexts/uid/a55d34bf-a6e5-4328-9af2-8d92d8280da5 HTTP/1.1" 200 OK
INFO:     127.0.0.1:38688 - "GET /contexts/uid/2023-06-21T16%3A18%3A07-game-YtlwNn HTTP/1.1" 200 OK
INFO:     127.0.0.1:38688 - "GET /contexts/uid/2023-06-21T16%3A39%3A34-game-3oWwPj HTTP/1.1" 200 OK
INFO:     127.0.0.1:46862 - "GET /contexts/uid/2023-06-21T17%3A57%3A46-game-9Ee4tF HTTP/1.1" 200 OK
INFO:     127.0.0.1:56942 - "GET /contexts/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:56942 - "GET /contexts/uid/2023-06-21T17%3A57%3A46-game-9Ee4tF HTTP/1.1" 200 OK
INFO:     127.0.0.1:35560 - "GET /contexts/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:35560 - "GET /contexts/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:35570 - "GET /contexts/uid/2023-06-21T16%3A18%3A07-game-YtlwNn HTTP/1.1" 200 OK
INFO